<a href="https://colab.research.google.com/github/Shrey-Viradiya/PyTorch_for_DL/blob/master/PyTorch_Simple_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Oct  1 07:23:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [4]:
X, y = load_boston(return_X_y=True)
X, y = shuffle(X,y, random_state = 259)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=259)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=259)

X_train = torch.from_numpy(X_train).to(device, dtype=torch.float32)
X_test = torch.from_numpy(X_test).to(device, dtype=torch.float32)
X_valid = torch.from_numpy(X_valid).to(device, dtype=torch.float32)
y_train = torch.from_numpy(y_train)
y_train = torch.squeeze(y_train).to(device, dtype=torch.float32)
y_test = torch.from_numpy(y_test)
y_test = torch.squeeze(y_test).to(device, dtype=torch.float32)
y_valid = torch.from_numpy(y_valid)
y_valid = torch.squeeze(y_valid).to(device, dtype=torch.float32)

## Creating a Network

In [6]:
class BostonModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.f1 = torch.nn.Linear(13,512)
        self.f2 = torch.nn.Linear(512,64)
        self.f3 = torch.nn.Linear(64,4)
        self.f4 = torch.nn.Linear(4,1)

    def forward(self, x):
        x = torch.nn.functional.relu(self.f1(x))
        x = torch.nn.functional.relu(self.f2(x))
        x = torch.nn.functional.relu(self.f3(x))
        x = self.f4(x)
        return x

In [7]:
bst = BostonModel()
bst.to(device)

BostonModel(
  (f1): Linear(in_features=13, out_features=512, bias=True)
  (f2): Linear(in_features=512, out_features=64, bias=True)
  (f3): Linear(in_features=64, out_features=4, bias=True)
  (f4): Linear(in_features=4, out_features=1, bias=True)
)

## Optimizer and Loss

In [8]:
optimizer = torch.optim.SGD(bst.parameters(), lr = 0.1)
loss_fun = torch.nn.MSELoss()

## Training

In [9]:
epochs = 200
batch_size = 32
steps_per_batch = X_train.shape[0] // batch_size

bst.train()
for epoch in range(epochs):
    training_loss = 0.0
    for step in range(steps_per_batch):
        curr_bat = torch.randint(X_train.shape[0], (batch_size,))
        data = X_train[curr_bat]
        tar = y_train[curr_bat]
        
        y_pred = bst(data)
        y_pred = torch.squeeze(y_pred)
        loss = loss_fun(y_pred, tar)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_loss += loss.data.item()
    training_loss /= steps_per_batch

    with torch.no_grad():
        output = bst(X_valid)
        output = torch.squeeze(output)
        loss = loss_fun(output, y_valid)
        validation_loss = loss.data.item()
    print(f'\repoch: {epoch} \t\t Training loss: {training_loss:8.3f} \t\t Validation loss: {validation_loss:8.3f} \t')
    

epoch: 0 		 Training loss:  231.623 		 Validation loss:  114.963 	
epoch: 1 		 Training loss:   61.669 		 Validation loss:  106.757 	
epoch: 2 		 Training loss:   85.396 		 Validation loss:  105.882 	
epoch: 3 		 Training loss:   71.169 		 Validation loss:  105.886 	
epoch: 4 		 Training loss:   81.216 		 Validation loss:  107.473 	
epoch: 5 		 Training loss:   72.324 		 Validation loss:  108.246 	
epoch: 6 		 Training loss:   83.788 		 Validation loss:  105.674 	
epoch: 7 		 Training loss:   75.437 		 Validation loss:  105.926 	
epoch: 8 		 Training loss:   73.481 		 Validation loss:  105.553 	
epoch: 9 		 Training loss:   87.607 		 Validation loss:  105.966 	
epoch: 10 		 Training loss:   69.937 		 Validation loss:  108.368 	
epoch: 11 		 Training loss:   83.650 		 Validation loss:  107.942 	
epoch: 12 		 Training loss:   85.712 		 Validation loss:  105.938 	
epoch: 13 		 Training loss:   60.278 		 Validation loss:  107.146 	
epoch: 14 		 Training loss:   79.396 		 Validation loss:  

In [10]:
with torch.no_grad():
    output = bst(X_test)
    output = torch.squeeze(output)
    loss = loss_fun(output, y_test)
    test_loss = loss.data.item()

print(f'Test loss: {test_loss:.3f}')

Test loss: 84.474
